<a href="https://colab.research.google.com/github/haidaramumca/PySpark/blob/main/Linear_Regression_Consulting_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 198 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ae22046b2db4fe06b24aa6ff0fc9bec79a5f7c000369d8ed605d382fd3d727c7
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CruiseInfo').getOrCreate()
from pyspark.ml.regression import LinearRegression
data = spark.read.csv("/content/cruise_ship_info.csv",inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [32]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')

In [37]:
indexed = indexer.fit(data).transform(data)

In [34]:
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [8]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [35]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [38]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers', 'length','cabins','passenger_density','cruise_cat'],outputCol='features')

In [40]:
output = assembler.transform(indexed)

In [41]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [42]:
final_data = output.select('features','crew')

In [43]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [44]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [45]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               112|
|   mean|  7.86375000000001|
| stddev|3.3519568373771826|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [46]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                46|
|   mean| 7.624782608695651|
| stddev|3.8817760763916267|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [47]:
lr = LinearRegression(labelCol='crew')

In [48]:
lr_model = lr.fit(train_data)

In [49]:
test_results = lr_model.evaluate(test_data)

In [50]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.45921009161732584|
| -1.3875227709482427|
| -1.0012787395692033|
| 0.09952060011873698|
| -0.6938584913211834|
| -1.4244485883336573|
| -0.3839014856453895|
| 0.33448127163688035|
| -0.7117868408502055|
|-0.01853360514712321|
|   1.761973408874148|
|  0.9153735151719928|
|-0.13675925552796286|
|-0.26880958060510984|
|   0.836969397866854|
|   0.836969397866854|
| 0.21606646794009343|
|-0.13026850306246285|
|  -1.396320415346512|
| -0.2923172413886732|
+--------------------+
only showing top 20 rows



In [51]:
test_results.rootMeanSquaredError

0.960985061766972

In [52]:
test_results.r2

0.9373504965577569

In [53]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [55]:
unlabled_data = test_data.select('features')

In [56]:
predictions = lr_model.transform(unlabled_data)

In [57]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|20.540789908382674|
|[5.0,86.0,21.04,9...| 9.387522770948243|
|[6.0,30.276999999...| 4.551278739569203|
|[6.0,110.23899999...|11.400479399881263|
|[6.0,112.0,38.0,9...|11.593858491321184|
|[7.0,89.6,25.5,9....|11.294448588333657|
|[9.0,59.058,17.0,...|  7.78390148564539|
|[9.0,81.0,21.44,9...|  9.66551872836312|
|[9.0,90.09,25.01,...| 9.401786840850205|
|[9.0,116.0,26.0,9...|11.018533605147123|
|[10.0,151.4,26.2,...|10.768026591125851|
|[11.0,90.0,22.4,9...|10.084626484828007|
|[11.0,91.62700000...| 9.136759255527963|
|[12.0,90.09,25.01...|  8.94880958060511|
|[12.0,91.0,20.32,...| 9.153030602133146|
|[12.0,91.0,20.32,...| 9.153030602133146|
|[12.0,108.865,27....|10.783933532059907|
|[13.0,25.0,3.82,5...| 3.080268503062463|
|[13.0,63.0,14.4,7...| 6.706320415346512|
|[13.0,76.0,18.74,...| 8.792317241388673|
+--------------------+------------

In [61]:
from pyspark.sql.functions import corr
data.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [62]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

